# Scraping Risk Analysis Reports from FRONTEX' website

## Imports

In [3]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import time

## Scraping

In [ ]:
#setting up driver
driver=webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))

### Navigating Page

In [6]:
#access frontex' public register of documents
driver.get('https://prd.frontex.europa.eu')